In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
from collections import OrderedDict, namedtuple
from sqlalchemy import func, distinct, text, and_
import pandas as pd

import util
from util.year import year as current_year
from db import session
import model

In [18]:
evaluations = session.query(
    model.Wave,
    model.Task,    
    func.count(distinct(model.User.id))
).\
    join(model.Task.r_wave).filter(model.Wave.year == current_year.id).\
    join(model.Task.modules).\
    join(model.Module.evaluations).\
    join(model.Evaluation.r_user)

evaluations_per_task = evaluations.\
    group_by(model.Task).order_by(model.Wave.id, model.Task.id)

evaluations_per_task[20:25]

[(Hrubá síla, Variace s opakováním, 234),
 (Hrubá síla, Variace bez opakování, 198),
 (Hrubá síla, Generování podmnožin, 181),
 (Hrubá síla, Backtracking, 148),
 (Hrubá síla, Problém osmi dam, 102)]

In [19]:
successful_evaluations = evaluations_per_task.\
    filter(model.Evaluation.ok == True)

successful_evaluations[20:25]

[(Hrubá síla, Variace s opakováním, 233),
 (Hrubá síla, Variace bez opakování, 195),
 (Hrubá síla, Generování podmnožin, 176),
 (Hrubá síla, Backtracking, 145),
 (Hrubá síla, Problém osmi dam, 98)]

In [20]:
per_module = session.query(
    model.Evaluation.user.label('user_id'),
    func.count(model.Evaluation.id).label('eval_count'),
).\
    join(model.Evaluation.r_module).join(model.Module.r_task).\
    join(model.Task.r_wave).\
    filter(model.Wave.year == current_year.id).\
    group_by(model.Evaluation.user,
             model.Evaluation.module).subquery()

EVAL_LIMITS = [10, 30, 50]
problematic_tasks = {
    limit: evaluations.
            filter(model.Evaluation.ok == False).
            join(per_module, per_module.c.user_id == model.User.id).\
            group_by(model.Task).\
            filter(per_module.c.eval_count > limit)
    for limit in EVAL_LIMITS
}

for limit, users in problematic_tasks.items():
    print(limit, users.all()[20:25])

10 [(Nultá vlna, Prostorová složitost, 67), (Nultá vlna, Pokročilí plavci, 160), (Nultá vlna, Řazení programů, 261), (Nultá vlna, Analýza četnosti slov, 55), (Nultá vlna, Debugování a testování, 113)]
50 [(Nultá vlna, Pokročilí plavci, 11), (Nultá vlna, Řazení programů, 16), (Nultá vlna, Analýza četnosti slov, 5), (Nultá vlna, Debugování a testování, 11), (Nultá vlna, PEP8, 14)]
30 [(Nultá vlna, Pokročilí plavci, 40), (Nultá vlna, Řazení programů, 56), (Nultá vlna, Analýza četnosti slov, 17), (Nultá vlna, Debugování a testování, 28), (Nultá vlna, PEP8, 39)]
